### Imports

In [8]:
import pandas as pd
import numpy as np
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense, Dropout
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

### Load your data

In [12]:
file_url = 'data/data.csv'
data = pd.read_csv(file_url)
print(data.shape)
data.head()

(500, 10)


,Asset ID,Asset Type,Floor,Room,Installation Date,Manufacturer,Operational Time (hrs),Work Orders,Repairs,Last Serviced Date
0,1,Elevator,7,103,1/6/2020,Manufacturer_4,39313,0,1,6/19/2023
1,2,Plumbing System,6,109,6/11/2022,Manufacturer_2,20012,0,1,1/6/2023
2,3,Fire Alarm,3,104,7/24/2019,Manufacturer_4,21546,10,5,9/24/2022
3,4,Elevator,1,105,1/4/2021,Manufacturer_4,47038,1,1,5/23/2023
4,5,Plumbing System,2,104,11/28/2022,Manufacturer_1,20022,10,3,1/29/2023


In [13]:
reference_date = pd.Timestamp.now()

data['Installation Date'] = pd.to_datetime(data['Installation Date'], format='%m/%d/%Y')
data['Days Since Installation'] = abs((data['Installation Date'] - reference_date).dt.days)

data['Last Serviced Date'] = pd.to_datetime(data['Last Serviced Date'], format='%m/%d/%Y')
data['Days Since Last Serviced'] = abs((data['Last Serviced Date'] - reference_date).dt.days)

data.head(20)


,Asset ID,Asset Type,Floor,Room,Installation Date,Manufacturer,Operational Time (hrs),Work Orders,Repairs,Last Serviced Date,Days Since Installation,Days Since Last Serviced
0,1,Elevator,7,103,2020-01-06,Manufacturer_4,39313,0,1,2023-06-19,1350,90
1,2,Plumbing System,6,109,2022-06-11,Manufacturer_2,20012,0,1,2023-01-06,463,254
2,3,Fire Alarm,3,104,2019-07-24,Manufacturer_4,21546,10,5,2022-09-24,1516,358
3,4,Elevator,1,105,2021-01-04,Manufacturer_4,47038,1,1,2023-05-23,986,117
4,5,Plumbing System,2,104,2022-11-28,Manufacturer_1,20022,10,3,2023-01-29,293,231
5,6,Fire Alarm,2,103,2021-07-29,Manufacturer_3,1602,9,4,2022-10-03,780,349
6,7,Fire Alarm,6,108,2020-08-23,Manufacturer_2,15650,2,3,2023-01-13,1120,247
7,8,HVAC,2,106,2021-06-03,Manufacturer_2,33932,1,4,2023-08-09,836,39
8,9,Fire Alarm,6,108,2023-07-30,Manufacturer_5,24332,5,4,2023-02-13,49,216
9,10,Elevator,7,101,2019-07-24,Manufacturer_4,18722,10,2,2023-05-29,1516,111


### Split the data into training and testing sets

In [16]:
features = ['Asset Type', 'Manufacturer', 'Operational Time (hrs)', 'Days Since Installation', 'Days Since Last Serviced']
target = ['Repairs']

X_train, X_test, Y_train, Y_test = train_test_split(data[features], data[target], test_size=0.2, random_state=42)

print(
    "Using %d samples for training and %d for validation"
    % (len(X_train), len(X_test))
)

Using 400 samples for training and 100 for validation


### Build a preprocessor for categorical and numerical data

In [19]:
num_features = ['Operational Time (hrs)', 'Repairs', 'Days Since Installation', 'Days Since Last Serviced']
cat_features = ['Asset Type', 'Manufacturer', 'manufacturer']

preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), num_features),
        ('cat', OneHotEncoder(), cat_features)])

### Preprocessing the training and testing data

In [22]:
Y_train.head()

,Repairs
249,0
433,0
19,5
322,3
332,1


In [20]:
X_train = preprocessor.fit_transform(X_train)
X_test = preprocessor.transform(X_test)

ValueError: A given column is not a column of the dataframe